# Script to extract SRTM elevations over study sites through Google Earth Engine (GEE)

Adapted from [2022 ICESat Hackweek notebook](https://github.com/ICESAT-2HackWeek/website2022/blob/main/book/tutorials/DataVisualization/Visualization_Earth_Engine_geemap_IS2_HW_2022.ipynb)

In [14]:
import os
import ee
import pandas as pd
import geopandas as gpd
import geemap
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt
import h5py
from rasterio import plot, warp

In [3]:
# -----GEE Authentication and Initialization
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

### AOI #1: Grand Mesa, CO

In [4]:
# -----Create geojson AOI
# used geojson.io
AOI1 = ee.Geometry({
        "type": "Polygon",
        "coordinates": [
          [
            [
              -108.24005126953124,
              38.884619201291905
            ],
            [
              -107.9,
              38.884619201291905
            ],
            [
              -107.9,
              39.11727568585598
            ],
            [
              -108.24005126953124,
              39.11727568585598
            ],
            [
              -108.24005126953124,
              38.884619201291905
            ]
          ]
        ]
      })

# -----Query for SRTM clip to AOI
SRTM1 = ee.Image("USGS/SRTMGL1_003").clip(AOI1)

# -----Plot elevations
# from: https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api
SRTM1_url = SRTM1.getThumbUrl({
    'min': 1000, 'max': 5000,
    'palette': ['white', 'green']})
Image(url=SRTM1_url)

### AOI #2: Mores Creek Summit, ID

In [5]:
# -----Create geojson AOI
# used geojson.io
AOI2 = ee.Geometry({
        "type": "Polygon",
        "coordinates": [
          [
            [
              -115.74131011962889,
              43.8855215890078
            ],
            [
              -115.61874389648438,
              43.8855215890078
            ],
            [
              -115.61874389648438,
              43.963661859597536
            ],
            [
              -115.74131011962889,
              43.963661859597536
            ],
            [
              -115.74131011962889,
              43.8855215890078
            ]
          ]
        ]
      })

# -----Query for SRTM clip to AOI
SRTM2 = ee.Image("USGS/SRTMGL1_003").clip(AOI2)

# -----Plot elevations
# from: https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api
SRTM2_url = SRTM2.getThumbUrl({
    'min': 1000, 'max': 5000,
    'palette': ['white', 'green']})
Image(url=SRTM2_url)

In [8]:
# -----Create a map
Map = geemap.Map()
Map.addLayer(AOI1)
Map.addLayer(AOI2)
Map.addLayer(SRTM2, {min:1000, max:3000})
Map.addLayer(SRTM1, {min:1000, max:3000})
Map.setCenter(-112.179660, 40.936019, zoom=6)
Map

Map(center=[40.936019, -112.17966], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

In [45]:
# -----Practice sampling... 
# Load some ICESat-2 data, for now we load one track, one beam (GT2R/L)
data_path = '/home/jovyan/data/'
is2_file = 'processed_ATL08_20200920013522_13240806_005_01.h5' # Sept = snow-off 

with h5py.File(data_path+is2_file, 'r') as f:
    is2_gt2l = pd.DataFrame(data={'lat': f['gt2l/land_segments/latitude'][:],
                                 'lon': f['gt2l/land_segments/longitude'][:],
                                  'h_te_best_fit': f['gt2l/land_segments/terrain/h_te_best_fit'][:]})

# select one of the beams and convert it to a geodataframe
gf = gpd.GeoDataFrame(is2_gt2l, geometry=gpd.points_from_xy(is2_gt2l.lon, is2_gt2l.lat), crs='epsg:7661')

# extract coordinates into numpy array
lat = np.array([x for x in gf['lat']], dtype=float)
lon = np.array([x for x in gf['lon']], dtype=float)
points = ee.Geometry.Point(lon[0], lat[0])

# sample SRTM at those coordinates
# SRTM1_sampled = SRTM1.sampleRegions(points)
# SRTM1_sampled

# SRTM1_info = SRTM1.getInfo()
# SRTM1_df = pd.DataFrame(SRTM1_info)
# SRTM1_df
# SRTM1.getInfo()

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.